# 장애인 콜택시 대기시간 예측 프로젝트
## 목표
- 기상 예보를 바탕으로, 장애인 콜택시 평균 대기 시간을 예측하기.
- 장애인 이동권 개선을 위하여서, 콜택시 대기시간을 예측하는 프로젝트이다.  
- 이를 통하여 장애인 콜택시를 이용하는 고객들의 불편사항을 개선하고 서비스의 품질을 높여 교통약자의 이동 편의 증진에 기여한다.
### 목차
1. 데이터 처리
2. 데이터 분석
3. 머신러닝 모델링
    - (나머지 3달 데이터는 테스트, 나머지는 학습데이터로 활용)
    - MAE, MAPE를 사용해 성능 높은 모델 선정
- 데이터 출처 : [서울시설공단_장애인콜택시 일별 이용현황](https://www.data.go.kr/data/15057705/openapi.do)  

## 라이브러리 불러오기

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 데이터 불러오기

In [3]:
file1 = pd.read_csv('./data/open_data.csv')
file2 = pd.read_csv('./data/weather.csv')

In [7]:
display(file1.head(2))
display(file2.head(2))

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611


,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
0,2012-01-01,0.4,-6.6,0.0,77.0,45.0,4.90
1,2012-01-02,-1.2,-8.3,0.0,80.0,48.0,6.16


# 1. 데이터 처리
- 기본 정보 확인
- 두 데이터프레임 날짜 기준으로 합치기.(단, file2의 경우 예보로 활용할 데이터이기 때문에 날짜를 하나 앞으로 당겨주어 데이터프레임을 합쳐준다.)

In [27]:
file1['time'] = pd.to_datetime(file1['기준일'])
file2['time'] = pd.to_datetime(file2['Date'].shift(-1))

In [28]:
display(file1.head(2))
display(file2.head(2))

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,time
0,2015-01-01,213,1023,924,23.2,2427,10764,2015-01-01
1,2015-01-02,420,3158,2839,17.2,2216,8611,2015-01-02


,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),time
0,2012-01-01,0.4,-6.6,0.0,77.0,45.0,4.90,2012-01-02
1,2012-01-02,-1.2,-8.3,0.0,80.0,48.0,6.16,2012-01-03


In [33]:
df = pd.merge(file1, file2, on='time', how='left') # 데이터 프레임 합치기
df.drop(['기준일','Date'], axis=1, inplace=True) # 필요 없는 열 제거 (file1의 '기준일', file2의 'Date' 열)
df = df.set_index(keys='time')
df = df.sort_index(ascending=True) # index 기준 정렬
df.head(2)

,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2)
time,,,,,,,,,,,,
2015-01-01,213,1023,924,23.2,2427,10764,-1.6,-1.6,0.0,64.0,64.0,0.00
2015-01-02,420,3158,2839,17.2,2216,8611,-4.3,-9.8,0.0,52.0,33.0,9.79


# 2. 데이터 분석

# 3. 머신러닝 모델링